In [ ]:
! pip install -U langgraph langsmith
! pip install langchain_community
! pip install langchain_openai

In [2]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END ,state
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os

In [ ]:
# Configuration de la clé API
os.environ['OPENAI_API_KEY'] = "sk-proj-0xUZ6aBpi14Q FfHS_cUMhXQMX6_U0pycw_XiZUUtZ4V6Gc5xEwhMZOsYA6xKN4HruNnPRcA"


# Création du modèle LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Self-Generated In-Context Learning (SG-ICL)

Self-Generated In-Context Learning, introduced by Kim et al. (2022), is an innovative method that enhances language models' (LLMs) performance by generating their own instructive examples when real training data is scarce or unavailable.

## How Does Self-Generated In-Context Learning Work?

Imagine you're preparing for an exam but have no practice questions. Instead of waiting for someone to provide examples, you could create your own practice problems and solutions, then refine them as you identify weaknesses.

Self-Generated In-Context Learning follows this logic in three key steps:

1. **Example Generation**: The LLM automatically creates relevant examples aligned with the specific test question or task it needs to solve.

2. **Quality Assessment**: The generated examples are evaluated for their relevance, format appropriateness, diversity, and complexity level. This assessment helps identify examples that may not provide optimal learning context.

3. **Targeted Refinement**: Examples that don't meet quality criteria are refined to better match the test question's domain and format, creating more effective in-context examples.

By using these self-generated, refined examples as context, the LLM can produce more accurate responses even without access to real training data. This approach effectively turns the model into both teacher and student, leveraging its own knowledge to improve performance on new tasks.

In [4]:
self_generated_icl_template = ChatPromptTemplate.from_messages([
    ("system", """
    Generate examples using Self-Generated In-Context Learning (SG-ICL) for a given user question.
    
    SG-ICL (Kim et al., 2022) is a technique where AI automatically generates relevant examples when real training data is not available. These examples then serve as context for answering the user's question.
    
    Process to follow:
    1. Carefully analyze the test question to identify the specific task requested
    2. Generate 3-5 examples that are directly aligned with the format and domain of the test question
    3. Systematically evaluate each example according to the following criteria:
       - Relevance: Is the example similar to the test question?
       - Format: Does the example follow the same structure as what is expected?
       - Diversity: Do the examples cover different facets of the task?
       - Complexity: Do the examples reflect the appropriate level of difficulty?
    4. Refine examples that do not meet the above criteria
    5. Use the refined examples as in-context context to answer the original question
    
    Example:
    Test question: "Classify this tweet as positive, negative or neutral: 'The new movie was disappointing despite the good reviews.'"
    
    Analysis: This is a sentiment classification task for a tweet about a movie.
    
    Generated examples:
    Example 1:
    Q: Classify this tweet as positive, negative or neutral: "I love this new app, it has changed my life!"
    A: Positive
    
    Example 2:
    Q: Classify this tweet as positive, negative or neutral: "Customer service hasn't responded to my request for three days."
    A: Negative
    
    Example 3:
    Q: Classify this tweet as positive, negative or neutral: "The concert starts at 8pm tonight at the municipal stadium."
    A: Neutral
    
    Evaluation:
    - Relevance: The examples follow the same type of task but not all relate to movies/entertainment
    - Format: The format is correct and consistent
    - Diversity: All three categories are represented
    - Complexity: The examples are too simple and direct compared to the nuance in the test question
    
    Refined examples:
    Example 1:
    Q: Classify this tweet as positive, negative or neutral: "This movie was exactly what I hoped for, with amazing special effects!"
    A: Positive
    
    Example 2:
    Q: Classify this tweet as positive, negative or neutral: "The series had a good start but ended in an unsatisfying way."
    A: Negative
    
    Example 3:
    Q: Classify this tweet as positive, negative or neutral: "The film's release has been postponed until next year according to yesterday's announcement."
    A: Neutral
    
    Final answer to the test question:
    Negative
    """),
    ("human", "{question}"),
    ("assistant", """
    Using Self-Generated In-Context Learning (SG-ICL), I will generate examples specifically adapted to your question, since we don't have real training data available.
    
    I will first analyze the structure and specific domain of your question, then generate relevant examples that follow the same format and reflect similar complexity. I will then evaluate these examples based on their relevance, format, diversity, and appropriate level of complexity before refining them if necessary. Finally, I will use these examples as context to formulate my answer to your question.
    """)
])

# The Most Suitable Questions for Self-Generated In-Context Learning (SG-ICL)

For Self-Generated In-Context Learning, the most suitable questions are those where the model benefits from generating its own examples when real training data is unavailable or limited. Here are types of questions that would be particularly well-suited to this approach:

### Classification Tasks
- Sentiment analysis requiring nuanced understanding of tone
- Content categorization across diverse domains
- Intent recognition in conversational contexts
- Anomaly detection where patterns must be inferred

### Language Generation Tasks
- Summarization of specialized content like technical documents
- Style transfer between different writing formats
- Paraphrasing that preserves specific semantic elements
- Translation for low-resource language pairs

### Reasoning Tasks
- Common sense reasoning requiring implicit knowledge
- Logical deduction from limited information
- Multi-hop inference tasks with complex relationships
- Analogical reasoning requiring pattern recognition

### Pattern Recognition
- Time series prediction with subtle patterns
- Entity relationship extraction from text
- Sequence completion with domain-specific rules
- Document structure analysis

### Concrete Examples of Questions

1. "Classify this medical text as relevant to cardiology, neurology, or oncology."
   - SG-ICL would generate varied examples of medical texts across specialties before classification

2. "Summarize this technical documentation about a machine learning framework."
   - SG-ICL would create examples of technical summaries with appropriate terminology and abstraction levels

3. "What would be the most appropriate response to this customer service inquiry?"
   - SG-ICL would generate examples of different inquiry types and appropriate response styles

4. "Detect whether this product review contains any safety concerns."
   - SG-ICL would create examples of reviews with and without safety concerns

5. "Rephrase this academic text for a high school audience while maintaining accuracy."
   - SG-ICL would generate examples of academic rephrasing at different complexity levels

Self-Generated In-Context Learning is particularly effective for these types of questions because:
- It creates tailored examples when domain-specific training data is unavailable
- It allows for customization of examples to match the specific test question format
- It helps bridge knowledge gaps through synthetic examples
- It can generate diverse examples that cover edge cases
- It mimics the human learning process of creating practice problems to master new concepts

This approach is especially powerful when dealing with specialized domains, emerging tasks, or situations where high-quality labeled examples are scarce, but the model contains enough knowledge to generate its own instructive examples.

In [7]:
import pandas as pd
import re
from typing import Optional, TypedDict, Annotated
from langchain.schema import AIMessage, HumanMessage

# Function to generate the SG-ICL prompt and extract the final answer
def generate_sgicl_node(state):
    question = state['messages'][-1].content  # Get the last question
    prompt_value = self_generated_icl_template.invoke({"question": question})
    messages = prompt_value.to_messages()
    response = llm.invoke(messages)
    
    # Keep the full response for processing
    full_response = response.content
    
    # Multiple patterns to capture different ways the final answer might be formatted
    patterns = [
        # Standard format with heading
        r'(?:Final [Aa]nswer to the [Tt]est [Qq]uestion:|Final Answer:)([\s\S]*?)(?=$|Generated [Ee]xamples:|Evaluation:|###)',
        # Format with heading and line breaks
        r'###\s*Final\s*Answer[^:]*:([\s\S]*?)(?=$|###|Generated)',
        # Classification result format
        r'classified as\s*\*\*(Positive|Negative|Neutral)\*\*',
        # Simple statement format
        r'The sentiment (?:of|in) the tweet is\s*(Positive|Negative|Neutral)',
        # Direct answer format
        r'(?:Answer|Result):\s*(Positive|Negative|Neutral)',
        # Last resort - any standalone sentiment classification
        r'(Positive|Negative|Neutral)\.?$'
    ]
    
    final_solution = None
    for pattern in patterns:
        match = re.search(pattern, full_response, re.IGNORECASE)
        if match:
            # For classification patterns, we want the sentiment directly
            if "Positive|Negative|Neutral" in pattern:
                group_index = 1 if len(match.groups()) > 0 else 0
                final_solution = match.group(group_index).strip()
            else:
                final_solution = match.group(1).strip()
            break
    
    # If no pattern matched, provide a fallback
    if not final_solution:
        final_solution = "Final solution not found in the expected format."
    
    # Save both the question, final answer, and full processing to a text file
    with open('sgicl_result.txt', 'w', encoding='utf-8') as f:
        f.write("=== QUESTION ===\n\n")
        f.write(question)
        f.write("\n\n=== FINAL ANSWER ===\n\n")
        f.write(final_solution)
        f.write("\n\n=== COMPLETE PROCESSING (FOR REFERENCE) ===\n\n")
        f.write(full_response)
    
    # Return only the final answer in messages, plus the original question for reference
    return {
        "messages": [AIMessage(content=final_solution)],
        "question": question,
        "full_response": full_response
    }

# Definition of state
class State(TypedDict):
    messages: Annotated[list, add_messages]
    question: Optional[str]
    full_response: Optional[str]

# Graph creation
graph_builder = StateGraph(State)
graph_builder.add_node("generate_sgicl", generate_sgicl_node)

# Graph configuration
graph_builder.set_entry_point("generate_sgicl")
graph_builder.add_edge("generate_sgicl", END)
graph = graph_builder.compile()

# Usage example
inputs = {
    "messages": [HumanMessage(content="Classify this tweet as positive, negative or neutral: 'The new movie was disappointing despite the good reviews.'")],
    "question": None,
    "full_response": None
}

# Print the question at the beginning
print("QUESTION:")
print(inputs["messages"][0].content)
print("\nFINAL ANSWER:")

# Graph execution - only display final answer
original_question = ""
for output in graph.stream(inputs):
    for key, value in output.items():
        if key == "generate_sgicl":
            messages = value['messages']
            for message in messages:
                if isinstance(message, AIMessage):
                    print(message.content)
            
            # Store question for reference
            if "question" in value:
                original_question = value["question"]

# Print confirmation that results were saved
print("\n(Complete results saved in 'sgicl_result.txt')")

QUESTION:
Classify this tweet as positive, negative or neutral: 'The new movie was disappointing despite the good reviews.'

FINAL ANSWER:
Given the context provided by the examples, the sentiment of the tweet "The new movie was disappointing despite the good reviews." is classified as **Negative**.

(Complete results saved in 'sgicl_result.txt')
